In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sym

Let $N \in \mathbb{N} $ , consider the Bellman equation :
$$
V(n) = \begin{cases}
A(n) \big[ V(n+N) + f_X(n) \big] + B(n) \big[  V(n-1) + S(n) f_Y(n) \big] 
& \text{if} \; 0< n \leq N 
\\
A(n) \big[ V(n+1) + f_X(n) \big] + B(n) \big[  V(n-N) + S(n) f_Y(n) \big]
& \text{if} \; N+1 \leq n < 2N+1 
\end{cases}
$$
or in the matrix form :
$$
\begin{pmatrix}
V(1) \\
\vdots \\
V(n_1) \\
\vdots \\
V(n_2) \\
\vdots \\
V(2N)
\end{pmatrix}
=
\begin{pmatrix}
1      &    0    & \ldots &   0   &  \ldots&   0    & 0 \\
\vdots \\
  0    & \ldots  &  \overbrace{B(n)}^{n_1-1\; \text{th column}} & \ldots & 
  \overbrace{A(n)}^{n_1 +N \;\text{th column} } & \ldots & 0 \\ 
\vdots \\
  0    & \ldots  &  \ldots &  \overbrace{B(n)}^{n_2-N\; \text{th column}} &
 \ldots &  \overbrace{A(n)}^{n_2+1 \;\text{th column} } &  0 \\ 
\vdots \\
  0    &    0    &  0    &   0    &  0  &   0     &  1 
\end{pmatrix}
\begin{pmatrix}
V(1) \\
\vdots \\
V(n_1) \\
\vdots \\
V(n_2) \\
\vdots \\
V(2N)
\end{pmatrix}
+
\begin{pmatrix}
0\\
\vdots \\
A(n_1)  f_X(n_1) + B(n_1) S(n_1) f_Y(n_1) \\
\vdots \\
A(n_2)  f_X(n_2) + B(n_2) S(n_2) f_Y(n_2) \\
\vdots \\
0
\end{pmatrix}
$$
and the boundary conditions :
$$
V(2N) = W_u \quad , \quad V(1) = W_l
$$

In [2]:
def Matrix(N,A,B):
    Matrix = np.zeros((2*N+2,2*N+2))
   
    # rows 0 ~ 2N+1 
    for n in range(2*N+1):
        #boundary
        if n == 0 :
            Matrix[n,0] = 1

        elif n == 2*N +1 :
            Matrix[n,2*N+1] = 1
        
        
        elif n > 0 and n <= N :
            
            #columns 0 ~ 2N+1
            for m in range(2*N):
                if m == n-1:
                    Matrix[n,m] = B(n)
                elif m== n+N:
                    Matrix[n,m] = A(n)
        elif n > N and n <= 2*N :
            
            #columns 0 ~ 2N+1
            for m in range(2*N):
                if m == n-N:
                    Matrix[n,m] = B(n)
                elif m== n+1:
                    Matrix[n,m] = A(n)
    return Matrix


In [3]:
def V(N,A,B,S,f_X,f_Y):
    Matrix = Matrix(N,A,B) -np.identity(2*N+0)
    vector = - (A * f_X + B * S * f_Y )
    return np.linalg.solve(Matrix , vector)

## Computation of $A(n)$ and $B(n)$
Compute $\mathbb{E}_{(S^*,M)}(e^{-r\tau} \; ; \; S^*_\tau = S^*_0 e^\delta) $
and $\mathbb{E}_{(S^*,M)}(e^{-r\tau} \; ; \; S^*_\tau = S^*_0 e^{-\delta} )$

In [63]:
# check martingale computation
k = sym.symbols('k')
X = sym.symbols('X')
r = sym.symbols('r')
p = sym.symbols('p')
d = sym.symbols('delta')
a = sym.symbols('a')
a_p = sym.log( (sym.exp(r) + sym.sqrt( sym.exp(2*r) -4*p*(1-p) )  )/(2*p)  )/d
a_m = sym.log( (sym.exp(r) - sym.sqrt( sym.exp(2*r) -4*p*(1-p)  ) )/(2*p)  )/d
L = sym.exp(-r)*( p*sym.exp(a*(X+d)-r*k ) + 
             (1-p)*sym.exp(a*(X-d)-r*k )                          )
R = sym.exp( a*X - r*k)
#result = Conditional - Exact

In [70]:
sym.simplify(L.subs(a,a_p)/R.subs(a,a_p))-1 , sym.simplify(L.subs(a,a_m)/R.subs(a,a_m))-1

(0, 0)

In [76]:
x = sym.symbols('x')
theta = sym.exp(a*x)
theta_p = theta.subs(a,a_p)
theta_m = theta.subs(a,a_m)
A = (theta_p.subs(x,x+k+1) - theta_m.subs(x,x+k+1) )/(
    theta_p.subs(x,2*(k+1)) - theta_m.subs(x,2*(k+1))
)
B = ( theta_p.subs(x,x-k-1)- theta_m.subs(x,x-k-1) )/(
    theta_p.subs(x,-2*k-2)- theta_m.subs(x,-2*k-2)
)

In [87]:
sym.simplify(A*theta_p.subs(x,k+1) + B*theta_p.subs(x,-k-1) -theta_p) 

0

In [88]:
sym.simplify(A*theta_m.subs(x,k+1) + B*theta_m.subs(x,-k-1) -theta_m)

0

In [ ]:
def enumerate(n,N,k):
    if 